# Making Moves: Chicago to Seattle

Moving to another city is always exciting but also comes with a lot of decisions. One of the biggest decisions is: where should I live?  
The area you live can greatly impact your first impression of the new city, not to mention it's a decision you HAVE to make before you move!  
Some people lived in busy places and would like a change of pace when they move. Others would like the same energy they lived around in their old city! This project is to help compare certain neighbourhoods between two cities (in this case, Chicago and Seattle) and help see which areas are similar and which ones might be the complete opposite.  

After doing some neighbourhood research, we will compare the neighbourhood to move from, Wicker Park in Chicago, to Capitol Hill in Seattle and then also Queen Anne in Seattle.

The data we will be using will be venue data from Foursquare. The venues we will be gathering will relate to many things that a person would be looking for when living, for example:  
    - Grocery stores
    - Affordability of these venues
    - Restaurants
    - Shopping Centers
...and more!  
  
Capitol Hill and Queen Anne will be the two neighbourhoods we will say our client is looking to move into from Wicker Park in Chicago. Let's see if we can help them decide.

### Import libraries

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim # latitude/longitude converter

import requests
from pandas.io.json import json_normalize #json files to pandas dataframe

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

print('Libraries imported.')

Libraries imported.


### Set up credentials

In [3]:
CLIENT_ID = 'VHZM10UIGEKH1UMU2RG0E13CGHNYPUFBVQ3QU2DHBO0VA2QF'
CLIENT_SECRET = 'JGRJYABMNMV2LRIR3YOVVALEE2UBS5PBXEOIMMBWIT5HWLTU'
VERSION = '20180604'

print('Client ID: ' + CLIENT_ID)
print('Client Secret: ' + CLIENT_SECRET)
print('Version: '+ VERSION)

Client ID: VHZM10UIGEKH1UMU2RG0E13CGHNYPUFBVQ3QU2DHBO0VA2QF
Client Secret: JGRJYABMNMV2LRIR3YOVVALEE2UBS5PBXEOIMMBWIT5HWLTU
Version: 20180604


# Methodology Section

### Import Chicago data from Foursquare API and create DataFrame

In [4]:
c_address = 'Wicker Park, Chicago, IL'

c_geolocator = Nominatim(user_agent='foursquare_agent')
c_location = c_geolocator.geocode(c_address)
c_latitude = c_location.latitude
c_longitude = c_location.longitude
print('Wicker Park, Chicago, IL coordinates: ', c_latitude, c_longitude)

Wicker Park, Chicago, IL coordinates:  41.9078018 -87.6781551


In [5]:
radius = 500
LIMIT = 100

c_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        c_latitude,
        c_longitude,
        VERSION,
        radius,
        LIMIT
        )

print(c_url)

https://api.foursquare.com/v2/venues/explore?client_id=VHZM10UIGEKH1UMU2RG0E13CGHNYPUFBVQ3QU2DHBO0VA2QF&client_secret=JGRJYABMNMV2LRIR3YOVVALEE2UBS5PBXEOIMMBWIT5HWLTU&ll=41.9078018,-87.6781551&v=20180604&radius=500&limit=100


In [6]:
results = requests.get(c_url).json()

In [7]:
# Retrieve category type from the json format
def get_category_type(row):
    try:
        category_list = row['categories']
    except:
        category_list = row['venue.categories']
    
    if len(category_list) == 0:
        return None
    else:
        return category_list[0]['name']

In [8]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) #flattens JSON

filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

In [9]:
nearby_venues.head(10)

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,Dove's Luncheonette,541c39ff498ee385598bdfff,Diner,41.909524,-87.677447
1,Big Star,4adbf2bbf964a520242b21e3,Taco Place,41.909229,-87.677279
2,The Violet Hour,49ba99c9f964a5207e531fe3,Speakeasy,41.909048,-87.677631
3,Wicker Park,4ae49d01f964a520519c21e3,Park,41.907960,-87.677267
4,Stan's Donuts & Coffee,52f2eebd498ea23d1a8d16d5,Donut Shop,41.909927,-87.677462
5,La Colombe Torrefaction,51d0683c498e3eb7325145da,Coffee Shop,41.909859,-87.677651
6,Myopic Books,4a294136f964a52075951fe3,Bookstore,41.909977,-87.676851
7,Taxim,4a345e4cf964a520249c1fe3,Greek Restaurant,41.909823,-87.676658
8,Bonci Pizzeria,5ada9afe41868621d2cce37b,Pizza Place,41.910194,-87.677409
9,Jeni's Splendid Ice Creams,54eb6f69498e75f95d61b00a,Ice Cream Shop,41.908827,-87.674944


In [10]:
nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
venue.name            100 non-null object
venue.id              100 non-null object
venue.categories      100 non-null object
venue.location.lat    100 non-null float64
venue.location.lng    100 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.0+ KB


100 venues came back in the results for Wicker Park area.

Just from looking at a few places on the list, we can see that there is quite a bit of variations. There is also a park near by which is a nice additional feature.

In [11]:
# Clean up column names
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

In [12]:
nearby_venues_map = nearby_venues[0:51]

In [99]:
#Generate map centered around address
wp_venues_map = folium.Map(location=[c_latitude, c_longitude], zoom_start=13)

#red marker added for main location
folium.features.CircleMarker(
    [c_latitude,c_longitude],
    radius=10,
    color='red',
    popup='Wicker Park',
    fill=True,
    fill_color= 'red',
    fill_opacity = 0.6
).add_to(wp_venues_map)

#blue markers for search results
for lat, lng, label in zip(nearby_venues_map.lat, nearby_venues_map.lng, nearby_venues_map.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(wp_venues_map)
    
wp_venues_map

### Explore Wicker

In [12]:
nearby_venues.count()

name          100
id            100
categories    100
lat           100
lng           100
dtype: int64

There are 100 venues within a radius of 500 meters of the center of Wicker Park in Chicago.

In [13]:
wp_unique = nearby_venues['categories'].unique()
nearby_venues['categories'].nunique()

71

There are 71 different types of venues around the area. That is pretty good!

In [14]:
top_categories = nearby_venues.groupby('categories').count()['name'].sort_values(ascending=False).head(10)
top_categories

categories
Pizza Place          6
Boutique             4
Clothing Store       3
Shoe Store           3
Coffee Shop          3
Bookstore            3
Bar                  3
Ramen Restaurant     2
Lounge               2
Accessories Store    2
Name: name, dtype: int64

The category with the most venues around is a pizza place, followed by a few different types of clothing related stores.

### Wicker Park Price Group and Likes

Let's take a look at 10 of the venues and its pricing group and likes to get a sense of the area.

In [61]:
def get_venue_info(dataframe_col):
    venue_info = []
    for venue_id in dataframe_col[0:10]:
        v_url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
        v_results = requests.get(v_url).json()
        try:
            venue_name = v_results['response']['venue']['name']
        except:
            venue_name = 'Not Available'
        try:
            venue_price = v_results['response']['venue']['price']
        except:
            venue_price = 'Not Available'
        try:
            venue_likes = v_results['response']['venue']['likes']['count']
        except:
            venue_likes = 'Not Available'
        venue_info.append([venue_name, venue_price, venue_likes])
    return venue_info

In [62]:
wicker_venue_info = get_venue_info(nearby_venues['id'])
print(wicker_venue_info)

[["Dove's Luncheonette", {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 385], ['Big Star', {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 1552], ['The Violet Hour', {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 669], ['Wicker Park', 'Not Available', 210], ["Stan's Donuts & Coffee", {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 566], ['La Colombe Torrefaction', {'tier': 1, 'message': 'Cheap', 'currency': '$'}, 176], ['Myopic Books', 'Not Available', 259], ['Taxim', {'tier': 3, 'message': 'Expensive', 'currency': '$'}, 87], ['Bonci Pizzeria', {'tier': 1, 'message': 'Cheap', 'currency': '$'}, 35], ["Jeni's Splendid Ice Creams", {'tier': 1, 'message': 'Cheap', 'currency': '$'}, 324]]


In [63]:
# nearby_venues[nearby_venues['id'] == '541c39ff498ee385598bdfff']

In [64]:
wicker_df = pd.DataFrame(wicker_venue_info, columns=['name', 'price_tier', 'likes'])
wicker_df

,name,price_tier,likes
0,Dove's Luncheonette,"{'tier': 2, 'message': 'Moderate', 'currency':...",385
1,Big Star,"{'tier': 2, 'message': 'Moderate', 'currency':...",1552
2,The Violet Hour,"{'tier': 2, 'message': 'Moderate', 'currency':...",669
3,Wicker Park,Not Available,210
4,Stan's Donuts & Coffee,"{'tier': 2, 'message': 'Moderate', 'currency':...",566
5,La Colombe Torrefaction,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",176
6,Myopic Books,Not Available,259
7,Taxim,"{'tier': 3, 'message': 'Expensive', 'currency'...",87
8,Bonci Pizzeria,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",35
9,Jeni's Splendid Ice Creams,"{'tier': 1, 'message': 'Cheap', 'currency': '$'}",324


In [65]:
def get_price_tier(df):
    price_tier = df['price_tier']
    
    if len(price_tier) == 0:
        return None
    elif price_tier == 'Not Available':
        return None
    else:
        return price_tier['message']

In [66]:
wicker_df['price_tier'] = wicker_df.apply(get_price_tier, axis=1)

In [67]:
wicker_df.sort_values('likes', ascending=False)

,name,price_tier,likes
1,Big Star,Moderate,1552
2,The Violet Hour,Moderate,669
4,Stan's Donuts & Coffee,Moderate,566
0,Dove's Luncheonette,Moderate,385
9,Jeni's Splendid Ice Creams,Cheap,324
6,Myopic Books,None,259
3,Wicker Park,None,210
5,La Colombe Torrefaction,Cheap,176
7,Taxim,Expensive,87
8,Bonci Pizzeria,Cheap,35


From these 10 venues, it would seem the average price of a store around the Wicker Park area is in the moderate-cheap range. Out of the 10, only one venue is labeled as expensive. The amount of 'likes' for these places aren't too low suggesting they are not rated poorly either.

### Import Queen Anne in Seattle data from Foursquare API

In [26]:
qa_address = 'Queen Anne, Seattle, WA'

qa_geolocator = Nominatim(user_agent='foursquare_agent')
qa_location = qa_geolocator.geocode(qa_address)
qa_lat = qa_location.latitude
qa_lng = qa_location.longitude

print('Queen Anne coordinates: ', qa_lat, qa_lng)

Queen Anne coordinates:  47.6394805 -122.3607453


In [27]:
qa_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        qa_lat,
        qa_lng,
        VERSION,
        radius,
        LIMIT
        )

print(qa_url)

https://api.foursquare.com/v2/venues/explore?client_id=VHZM10UIGEKH1UMU2RG0E13CGHNYPUFBVQ3QU2DHBO0VA2QF&client_secret=JGRJYABMNMV2LRIR3YOVVALEE2UBS5PBXEOIMMBWIT5HWLTU&ll=47.6394805,-122.3607453&v=20180604&radius=500&limit=100


In [28]:
qa_results = requests.get(qa_url).json()

In [30]:
qa_venues = qa_results['response']['groups'][0]['items']

qa_nearby_venues = json_normalize(qa_venues)

qa_nearby_venues = qa_nearby_venues.loc[:, filtered_columns]

In [94]:
qa_nearby_venues['venue.categories'] = qa_nearby_venues.apply(get_category_type, axis=1)
qa_nearby_venues.columns = [col.split('.')[-1] for col in qa_nearby_venues.columns]

qa_nearby_venues.head(10)

,name,id,categories,lat,lng
0,Blue Highway Games,4b107d07f964a5208e7123e3,Toy / Game Store,47.638564,-122.357133
1,Eden Hill,55fb84ad498ef87ed7839f63,Restaurant,47.638698,-122.357034
2,How To Cook A Wolf,49e82bc3f964a5204a651fe3,Italian Restaurant,47.638649,-122.356932
3,Haute Yoga,4a8e262df964a5201d1220e3,Yoga Studio,47.638106,-122.357024
4,Queen Anne Farmers Market,4bf5a9164d5f20a1a5dd97fe,Farmers Market,47.637064,-122.357692
5,Mezcaleria Oaxaca,4e8a6bdfcc215b0368100cb8,Mexican Restaurant,47.637899,-122.357230
6,Homegrown,4ccf7165de0f6dcbc05d6463,Sandwich Place,47.638472,-122.357048
7,Caffe Ladro,4244a600f964a5206e201fe3,Café,47.638644,-122.357090
8,Damask Tattoo,4bd3782bcaff95218c06d5f0,Cosmetics Shop,47.638626,-122.356702
9,Bite Box,5839ff83d7b47337a00eb8ca,Bistro,47.639493,-122.361345


In [95]:
qa_nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
name          50 non-null object
id            50 non-null object
categories    50 non-null object
lat           50 non-null float64
lng           50 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.0+ KB


50 venues around the area of Queen Anne.

In [81]:
#Generate map centered around address
qa_venues_map = folium.Map(location=[qa_lat, qa_lng], zoom_start=13)

#red marker added for main location
folium.features.CircleMarker(
    [qa_lat,qa_lng],
    radius=10,
    color='red',
    popup='Queen Anne',
    fill=True,
    fill_color= 'red',
    fill_opacity = 0.6
).add_to(qa_venues_map)

#blue markers for search results
for lat, lng, label in zip(qa_nearby_venues.lat, qa_nearby_venues.lng, qa_nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(qa_venues_map)
    
qa_venues_map

### Explore Queen Anne

In [32]:
qa_unique = qa_nearby_venues['categories'].unique()
qa_nearby_venues['categories'].nunique()

39

Queen Anne has 39 different types of venue.

In [33]:
qa_wp_same = []
qa_has = []
for x in qa_unique:
    if x in wp_unique:
        qa_wp_same.append(x)
    else:
        qa_has.append(x)
print('Queen Anne and Wicker Parker have: ', '\n', qa_wp_same)
print('\n')
print('Queen Anne have: ', '\n', qa_has)

wp_has = []
for x in wp_unique:
    if x in qa_unique:
        continue
    else:
        wp_has.append(x)
print('\n')
print('Wicker Park have: ', '\n', wp_has)

Queen Anne and Wicker Parker have:  
 ['Farmers Market', 'Mexican Restaurant', 'Cosmetics Shop', 'Bakery', 'Frozen Yogurt Shop', 'Coffee Shop', 'Wine Shop', 'Dive Bar', 'Gym / Fitness Center', 'Pizza Place', 'Gourmet Shop', 'Pharmacy', 'Mediterranean Restaurant', 'Spa', 'Playground', 'Art Gallery']


Queen Anne have:  
 ['Toy / Game Store', 'Restaurant', 'Italian Restaurant', 'Yoga Studio', 'Sandwich Place', 'Café', 'Bistro', 'Nail Salon', 'Grocery Store', 'Market', 'Smoothie Shop', 'Thai Restaurant', 'Pet Store', 'American Restaurant', 'Music Venue', 'Paper / Office Supplies Store', 'Pub', 'Cupcake Shop', 'Baseball Field', 'Seafood Restaurant', 'Bus Stop', 'Kids Store', 'Jewelry Store']


Wicker Park have:  
 ['Diner', 'Taco Place', 'Speakeasy', 'Park', 'Donut Shop', 'Bookstore', 'Greek Restaurant', 'Ice Cream Shop', 'Accessories Store', 'Boutique', 'French Restaurant', 'Video Game Store', 'Vegetarian / Vegan Restaurant', 'Bike Shop', 'Bar', 'Breakfast Spot', 'Sporting Goods Shop', 'B

Above we can see that there are a lot more different types of venues in the Wicker Park area compared to Queen Anne. Although some are the same types of venue except with different names (e.g. Nail Salon and Salon/Barbershop), there are a lot that are simply different. This suggests there are more variety in Wicker Park compared to Queen Anne.

In [34]:
qa_nearby_venues.groupby('categories').count()['name'].sort_values(ascending=False).head(10)

categories
Pizza Place           3
Bus Stop              2
Grocery Store         2
Mexican Restaurant    2
Coffee Shop           2
Playground            2
Restaurant            2
Yoga Studio           2
Bakery                2
Thai Restaurant       2
Name: name, dtype: int64

The categories of venues are listed above from most frequent to list. Here we list the top 10.

### Queen Anne Price Group and Likes

Again, we will check out the price group and likes of 10 venues near Queen Anne.

In [68]:
qa_venue_info = get_venue_info(qa_nearby_venues['id'])

In [69]:
qa_venue_info

[['Blue Highway Games', 'Not Available', 36],
 ['Eden Hill', {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 29],
 ['How To Cook A Wolf',
  {'tier': 2, 'message': 'Moderate', 'currency': '$'},
  138],
 ['Haute Yoga', 'Not Available', 15],
 ['Queen Anne Farmers Market', 'Not Available', 39],
 ['Mezcaleria Oaxaca',
  {'tier': 2, 'message': 'Moderate', 'currency': '$'},
  86],
 ['Homegrown', {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 70],
 ['Caffe Ladro', {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 41],
 ['Damask Tattoo', 'Not Available', 12],
 ['Bite Box', 'Not Available', 7]]

In [70]:
qa_df = pd.DataFrame(qa_venue_info, columns=['name', 'price_tier', 'likes'])
qa_df    

,name,price_tier,likes
0,Blue Highway Games,Not Available,36
1,Eden Hill,"{'tier': 2, 'message': 'Moderate', 'currency':...",29
2,How To Cook A Wolf,"{'tier': 2, 'message': 'Moderate', 'currency':...",138
3,Haute Yoga,Not Available,15
4,Queen Anne Farmers Market,Not Available,39
5,Mezcaleria Oaxaca,"{'tier': 2, 'message': 'Moderate', 'currency':...",86
6,Homegrown,"{'tier': 2, 'message': 'Moderate', 'currency':...",70
7,Caffe Ladro,"{'tier': 2, 'message': 'Moderate', 'currency':...",41
8,Damask Tattoo,Not Available,12
9,Bite Box,Not Available,7


In [71]:
qa_df['price_tier'] = qa_df.apply(get_price_tier, axis=1)
qa_df.sort_values('likes', ascending=False)

,name,price_tier,likes
2,How To Cook A Wolf,Moderate,138
5,Mezcaleria Oaxaca,Moderate,86
6,Homegrown,Moderate,70
7,Caffe Ladro,Moderate,41
4,Queen Anne Farmers Market,None,39
0,Blue Highway Games,None,36
1,Eden Hill,Moderate,29
3,Haute Yoga,None,15
8,Damask Tattoo,None,12
9,Bite Box,None,7


Queen Anne doesn't seem to have as many likes at their venues compared to Wicker Park. Also, there are a lot more activity venues that do not have a labeled price tier since they are not restaurants.

### Import Capitol Hill in Seattle Data From Foursquare API

In [49]:
address = 'Capitol Hill, Seattle, WA'

ch_geolocator = Nominatim(user_agent='foursquare_agent')
ch_location = ch_geolocator.geocode(address)
ch_lat = ch_location.latitude
ch_lng = ch_location.longitude

print('Capitol Hill coordinates : ', ch_lat, ch_lng)

Capitol Hill coordinates :  47.6238307 -122.3183689


In [50]:
ch_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        ch_lat,
        ch_lng,
        VERSION,
        radius,
        LIMIT
        )

print(ch_url)

https://api.foursquare.com/v2/venues/explore?client_id=VHZM10UIGEKH1UMU2RG0E13CGHNYPUFBVQ3QU2DHBO0VA2QF&client_secret=JGRJYABMNMV2LRIR3YOVVALEE2UBS5PBXEOIMMBWIT5HWLTU&ll=47.6238307,-122.3183689&v=20180604&radius=500&limit=100


In [51]:
ch_results = requests.get(ch_url).json()

In [52]:
ch_venues = ch_results['response']['groups'][0]['items']

ch_nearby_venues = json_normalize(ch_venues)

ch_nearby_venues = ch_nearby_venues.loc[:,filtered_columns]

ch_nearby_venues.head()

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,Espresso Vivace,48e4b935f964a52026521fe3,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",47.623667,-122.320692
1,CorePower Yoga,51b5d221498eead65cd90978,"[{'id': '4bf58dd8d48988d102941735', 'name': 'Y...",47.623323,-122.321225
2,Corvus & Co.,5747b464498efe1302c3e268,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",47.624380,-122.321141
3,Altura,4e8bd0f7cc2112f67533b390,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",47.624763,-122.321103
4,Witness,52156bbd498eb27e8d0eb520,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",47.622339,-122.320962


In [84]:
ch_nearby_venues['venue.categories'] = ch_nearby_venues.apply(get_category_type,axis = 1)
ch_nearby_venues.columns = [col.split('.')[-1] for col in ch_nearby_venues.columns]
ch_nearby_venues.head(10)

,name,id,categories,lat,lng
0,Espresso Vivace,48e4b935f964a52026521fe3,Coffee Shop,47.623667,-122.320692
1,CorePower Yoga,51b5d221498eead65cd90978,Yoga Studio,47.623323,-122.321225
2,Corvus & Co.,5747b464498efe1302c3e268,Cocktail Bar,47.624380,-122.321141
3,Altura,4e8bd0f7cc2112f67533b390,Italian Restaurant,47.624763,-122.321103
4,Witness,52156bbd498eb27e8d0eb520,Cocktail Bar,47.622339,-122.320962
5,Poppy,49e3eb68f964a520ec621fe3,Indian Restaurant,47.625015,-122.320749
6,Due' Cucina Italiana,58096b4238faf62190651af2,Italian Restaurant,47.622431,-122.320586
7,Boca Restobar & Grill,5a87713c7564f77ddfc281c2,Argentinian Restaurant,47.622617,-122.320778
8,Lionhead,55ca95a7498e3dfea082b08e,Szechuan Restaurant,47.624881,-122.320824
9,Herb & Bitter Public House,55751527498ef4e2722759e3,Cocktail Bar,47.623560,-122.320917


In [91]:
ch_nearby_venues.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 5 columns):
name          62 non-null object
id            62 non-null object
categories    62 non-null object
lat           62 non-null float64
lng           62 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.5+ KB


62 venues for the area by Capitol Hill.

In [119]:
ch_nearby_venues_map = ch_nearby_venues[0:10]

62 total venues came back for the Capitol Hill area.

In [120]:
#Generate map centered around address
ch_venues_map = folium.Map(location=[ch_lat, ch_lng], zoom_start=13)

#red marker added for main location
folium.features.CircleMarker(
    [ch_lat,ch_lng],
    radius=10,
    color='red',
    popup='Capitol Hill',
    fill=True,
    fill_color= 'red',
    fill_opacity = 0.6
).add_to(ch_venues_map)

#blue markers for search results
for lat, lng, label in zip(ch_nearby_venues_map.lat, ch_nearby_venues_map.lng, ch_nearby_venues_map.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius = 5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(ch_venues_map)
    
ch_venues_map

### Explore Capitol Hill

In [54]:
ch_unique = ch_nearby_venues['categories'].unique()
ch_nearby_venues['categories'].nunique()

44

There are 44 unique types of venues around Capitol Hill.

In [55]:
ch_wp_same = []
ch_has = []
for x in ch_unique:
    if x in wp_unique:
        ch_wp_same.append(x)
    else:
        ch_has.append(x)
print('Capitol Hill and Wicker Parker have: ', '\n', ch_wp_same)
print('\n')
print('Capitol Hill have: ', '\n', ch_has)

wp_has = []
for x in wp_unique:
    if x in ch_unique:
        continue
    else:
        wp_has.append(x)
print('\n')
print('Wicker Park have: ', '\n', wp_has)

Capitol Hill and Wicker Parker have:  
 ['Coffee Shop', 'Cocktail Bar', 'Indian Restaurant', 'Argentinian Restaurant', 'Bar', 'Frozen Yogurt Shop', 'Gym / Fitness Center', 'Thrift / Vintage Store', 'Salon / Barbershop', 'Bookstore', 'Spa', 'Taco Place', 'Gym', 'Lounge', 'Bakery', 'Pharmacy', 'Mexican Restaurant', 'Furniture / Home Store', 'Korean Restaurant', 'Greek Restaurant', 'Clothing Store']


Capitol Hill have:  
 ['Yoga Studio', 'Italian Restaurant', 'Szechuan Restaurant', "Men's Store", 'Noodle House', 'Massage Studio', 'Thai Restaurant', 'Record Shop', 'Restaurant', 'American Restaurant', 'Café', 'Seafood Restaurant', 'Pet Store', 'Southern / Soul Food Restaurant', 'Mobile Phone Shop', 'Pub', 'Marijuana Dispensary', 'Ethiopian Restaurant', 'Gift Shop', 'Japanese Restaurant', 'Locksmith', 'Bed & Breakfast', 'ATM']


Wicker Park have:  
 ['Diner', 'Speakeasy', 'Park', 'Donut Shop', 'Pizza Place', 'Ice Cream Shop', 'Accessories Store', 'Farmers Market', 'Boutique', 'French Restau

Similar to the comparison between Wicker Park and Queen Anne, Capitol Hill has less variety of venues than Wicker Park. Some are the same type with different names (e.g. Pet Store and Pet Service), but Wicker Park still has more.

### Capitol Hill Price Group and Likes

Let's have a look at the venue information of the places in Capitol Hill!

In [72]:
ch_venue_info = get_venue_info(ch_nearby_venues['id'])

In [73]:
ch_venue_info

[['Espresso Vivace', {'tier': 1, 'message': 'Cheap', 'currency': '$'}, 379],
 ['CorePower Yoga', 'Not Available', 31],
 ['Corvus & Co.', {'tier': 3, 'message': 'Expensive', 'currency': '$'}, 45],
 ['Altura', {'tier': 4, 'message': 'Very Expensive', 'currency': '$'}, 79],
 ['Witness', {'tier': 2, 'message': 'Moderate', 'currency': '$'}, 210],
 ['Poppy', {'tier': 3, 'message': 'Expensive', 'currency': '$'}, 195],
 ["Due' Cucina Italiana",
  {'tier': 2, 'message': 'Moderate', 'currency': '$'},
  32],
 ['Boca Restobar & Grill',
  {'tier': 2, 'message': 'Moderate', 'currency': '$'},
  9],
 ['Lionhead', 'Not Available', 51],
 ['Herb & Bitter Public House',
  {'tier': 3, 'message': 'Expensive', 'currency': '$'},
  86]]

In [74]:
ch_df = pd.DataFrame(ch_venue_info, columns=['name', 'price_tier', 'likes'])
ch_df['price_tier'] = ch_df.apply(get_price_tier, axis=1)
ch_df.sort_values('likes', ascending=False)

,name,price_tier,likes
0,Espresso Vivace,Cheap,379
4,Witness,Moderate,210
5,Poppy,Expensive,195
9,Herb & Bitter Public House,Expensive,86
3,Altura,Very Expensive,79
8,Lionhead,None,51
2,Corvus & Co.,Expensive,45
6,Due' Cucina Italiana,Moderate,32
1,CorePower Yoga,None,31
7,Boca Restobar & Grill,Moderate,9


Seems like Capitol Hill have a lot more expensive restaurants than the other two areas! Similar to Wicker Parker, it has more restaurants in the area than activity venues(such as Yoga, parks, tattoo parlors, etc.)

# Results

##### Price Range

Based on the analysis, it would seem that Queen Anne has might have a closer price range of venues to Wicker Park than Capitol Hill. However, due to the limited amounts of premium calls per day, it is possible that the expensive restaurants in Queen Anne were simply missed. Queen Anne also had more venues that do not have a labeled price tier making it harder to compare as well.  
Capitol Hill definitely has the more expensive venues of the three areas. Just within the closer-examined 10 venues, it already had significantly more expensive venues than the other two locations.

##### Amount of Venues

Queen Anne has less venues than Wicker Park does. Capitol Hill also has less but more than Queen Anne.  
It would seem that Wicker Park outnumbers both areas in terms of different types of venue. In the two sections of seeing what the difference in the category labels of venues are, Wicker Park have a lot more to offer.

##### Popularity

Wicker Park venues have significantly more likes than Queen Anne or Capitol Hill.  
It signifies that Wicker Park could be a busier area with more foot traffic. Or people are very vocal in that area.  
Capitol Hill comes second.

# Discussion

After the anaylsis, it would seem that Capitol Hill would be the better choice for someone looking for a more similar experience they got in Wicker Park. It is a tad bit more expensive but it will also provide an entertaining lifestyle. If the individual appreciated the food choices in Wicker Park, Capitol Hill can offer a similar experience on a smaller scale.  

Queen Anne seems more calm and quiet. This would be a good choice for someone who would like to switch up what they got at Wicker Park. Although not as much variety of food types, there are many activities around.

# Conclusion

Although we got a glimpse of the neighbourhoods, this is not everything there is to it. In order to know the neighbourhood, it cannot simply be based upon the venues in the area. It is based on a number of other factors such as safety, accessibility, etc.  
Examining venues closer is important because moving across country can be stressful and frightening but when you are more comfortable with what you will be surrounded with, it can take some stress off.  
Everyone needs activities to do when they are in a new place and choosing an area that fits your need especially during the first year can really set the tone for the rest of your time in the new city. Start off on the right foot and do some research and comparison before moving!